
# Template — MVP: *Machine Learning & Analytics*
**Autor:** Bruno dos Santos Lopes

**Data:** 27/09/2025

**Matrícula:** 4052025000271

**Dataset:** [Store Sales - Time Series Forecasting](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data)

> **Importante:**
A estrutura base deste notebook pode servir como um guia inicial para desenvolver suas análises, já que contempla grande parte das sugestões do checklist apresentado no enunciado do MVP. Entretanto, é importante destacar que esta estrutura é apenas um ponto de partida: poderão ser necessárias etapas e análises adicionais além das aqui exemplificadas.

> O essencial é garantir profundidade nas discussões e análises, construindo um storytelling consistente que explore os principais conceitos e técnicas vistos nas aulas da Sprint de Machine Learning & Analytics.

> Lembre-se: não existe uma receita pronta. A ordem e as seções detalhadas abaixo são apenas sugestões. O problema escolhido e a história que você deseja contar devem guiar, em grande parte, a forma final do seu trabalho.

---



## ✅ Checklist do MVP (o que precisa conter)
- [ ] **Problema definido** e contexto de negócio
- [ ] **Carga e preparação** dos dados (sem vazamento de dados)
- [ ] **Divisão** em treino/validação/teste (ou validação cruzada apropriada)
- [ ] **Tratamento**: limpeza, transformação e **engenharia de atributos**
- [ ] **Modelagem**: comparar abordagens/modelos (com **baseline**)
- [ ] **Otimização de hiperparâmetros**
- [ ] **Avaliação** com **métricas adequadas** e discussão de limitações
- [ ] **Boas práticas**: seeds fixas, tempo de treino, recursos computacionais, documentação
- [ ] **Pipelines reprodutíveis** (sempre que possível)



## 1. Escopo, objetivo e definição do problema
**TODO:** Explique brevemente:
- Contexto do problema e objetivo:
O problema central abordado neste MVP é a ruptura de estoque, situação em que a demanda de um produto não pode ser atendida devido à falta de reposição adequada. Isso gera perda de vendas, redução da satisfação do cliente e ineficiências na cadeia logística.
O objetivo do MVP é desenvolver uma solução baseada em dados que permita prever a demanda e apoiar a reposição de produtos, reduzindo a probabilidade de ruptura.
Empresas de varejo precisam prever a demanda de produtos em diferentes lojas para otimizar estoque, reduzir perdas e maximizar o faturamento. O dataset Store Sales – Time Series Forecasting, disponibilizado pelo Kaggle, contém dados históricos de vendas diárias por loja e família de produtos, incluindo informações de promoções, feriados e indicadores externos.
O objetivo deste projeto é desenvolver modelos de previsão de vendas (forecasting) que permitam estimar a demanda futura por produto/loja, considerando tendências, sazonalidades e eventos externos.  

- Tipo de tarefa: Problema de previsão de séries temporais (forecasting)
- Natureza do problema: Regressão (variável-alvo = valor numérico de vendas).  
- Área de aplicação: Dados tabulares e temporais e Aplicação em Analytics e Inteligência de Negócios (BI/AI para varejo).  
- Gestão de estoque eficiente: evitar ruptura (falta de produtos) e excesso (desperdício e custos). Planejamento de promoções: prever impacto de descontos e campanhas sazonais. Alocação de recursos: otimizar logística, distribuição e compras junto a fornecedores. Suporte à tomada de decisão: fornecer previsões confiáveis para gestores de diferentes áreas (vendas, marketing, supply chain).



## 2. Reprodutibilidade e ambiente
Especifique o ambiente. Por exemplo:
- Bibliotecas usadas.
- Seeds fixas para reprodutibilidade.

In [ ]:
# === Setup básico e reprodutibilidade ===
import os, random, sys, math, time, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

# Scikit-learn (pré-processamento, modelagem, avaliação)
from sklearn.model_selection import (train_test_split, StratifiedKFold, KFold, TimeSeriesSplit, RandomizedSearchCV)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix,
    mean_absolute_error, mean_squared_error, r2_score,
    silhouette_score
)
from scipy.stats import randint, uniform

# Modelagem de séries temporais
import statsmodels.api as sm
from prophet import Prophet
import xgboost as xgb
import lightgbm as lgb

# Configurações globais
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# Para frameworks adicionais:
# import torch; torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
# import tensorflow as tf; tf.random.set_seed(SEED)

print("Python:", sys.version.split()[0])
print("Seed global:", SEED)


### 2.1 Funções python (opcional)
Defina, se necessário, funções em Python para reutilizar seu código e torná-lo mais organizado. Essa é uma boa prática de programação que facilita a leitura, manutenção e evolução do seu projeto.

In [ ]:
# Função para criar lags de séries temporais
def create_lag_features(df, group_cols, target_col, lags=[1,7,14,28]):
    """
    Cria colunas de lags para séries temporais agrupadas.

    Parâmetros:
    - df: DataFrame contendo a série temporal
    - group_cols: colunas para agrupar (ex.: ['store_nbr', 'family'])
    - target_col: coluna alvo (ex.: 'sales')
    - lags: lista de períodos de lag

    Retorno:
    - DataFrame com novas colunas de lags
    """
    df_copy = df.copy()
    for lag in lags:
        df_copy[f'{target_col}_lag_{lag}'] = df_copy.groupby(group_cols)[target_col].shift(lag)
    return df_copy

# Função para calcular métricas de previsão
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_forecast(y_true, y_pred):
    """
    Calcula RMSE, MAE e MAPE de uma previsão.

    Parâmetros:
    - y_true: valores reais
    - y_pred: valores previstos

    Retorno:
    - dicionário com métricas
    """
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    mape = (abs(y_true - y_pred) / (y_true + 1e-9)).mean() * 100  # evita divisão por zero
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

    # Função para plotar série temporal

    import matplotlib.pyplot as plt

def plot_series(df, date_col, target_col, title=None, figsize=(12,5)):
    """
    Plota série temporal.

    Parâmetros:
    - df: DataFrame
    - date_col: coluna de datas
    - target_col: coluna de valores
    - title: título opcional
    """
    plt.figure(figsize=figsize)
    plt.plot(df[date_col], df[target_col], marker='o', linestyle='-')
    plt.xlabel('Data')
    plt.ylabel(target_col)
    if title:
        plt.title(title)
    plt.grid(True)
    plt.show()


## 3. Dados: carga, entendimento e qualidade
O presente projeto utiliza o dataset Store Sales — Time Series Forecasting, disponibilizado no Kaggle, cujo objetivo é prever a demanda diária de produtos em diferentes lojas, permitindo otimizar estoques, reduzir perdas e apoiar decisões estratégicas de marketing e logística. Os dados foram carregados diretamente do GitHub, garantindo a execução imediata do notebook no Google Colab, sem necessidade de configuração adicional.

O dataset é composto por múltiplos arquivos CSV, cada um com informações complementares sobre vendas, lojas, feriados, eventos e preços de petróleo. O arquivo train.csv contém as vendas históricas diárias por loja e família de produtos, sendo a variável-alvo a coluna sales. O arquivo test.csv corresponde ao conjunto de teste, utilizado para submissão de previsões. O arquivo stores.csv fornece dados adicionais sobre as lojas, incluindo cidade, estado, tipo e cluster de classificação. O arquivo holidays_events.csv lista feriados nacionais e regionais, bem como eventos especiais, indicando se foram transferidos. O arquivo transactions.csv informa o número diário de transações por loja, e o arquivo oil.csv registra o preço diário do petróleo WTI, que pode impactar o comportamento de vendas de certos produtos. Por fim, o arquivo sample_submission.csv serve como template para submissão das previsões no Kaggle.

As principais variáveis presentes no dataset incluem: store_nbr (número da loja), family (categoria do produto), date (data da venda), sales (quantidade vendida), city, state, type e cluster das lojas, além de variáveis relacionadas a feriados (type, locale, locale_name, transferred), número de transações diárias e preço do petróleo (dcoilwtico).

Uma análise preliminar da qualidade dos dados indicou a presença de valores ausentes em algumas colunas, como o preço do petróleo, que será tratado via interpolação temporal. Zeros nas vendas podem indicar dias sem vendas ou feriados, e datas de feriados transferidos foram cuidadosamente consideradas para evitar inconsistências. Todas as transformações e criação de features de lag serão realizadas apenas com informações históricas, evitando vazamento de dados e garantindo a reprodutibilidade das previsões.

Do ponto de vista ético, os dados são públicos, voltados para fins educacionais e não contêm informações pessoais ou sensíveis. O uso das informações é restrito à análise de séries temporais de vendas e à construção de modelos de previsão de demanda, respeitando a confidencialidade e a integridade dos dados.


In [ ]:
#Carga dos dados
# URL base dos arquivos CSV no GitHub
base_url = "https://raw.githubusercontent.com/BrunoLopes011/EntregaMVP3/main/"

# Lista de arquivos
arquivos = [
    "holidays_events.csv",
    "oil.csv",
    "sample_submission.csv",
    "stores.csv",
    "test.csv",
    "train.csv",
    "transactions.csv"
]

# Leitura dos arquivos com pandas
print("📥 Lendo arquivos diretamente do GitHub...")
holidays_events = pd.read_csv(base_url + "holidays_events.csv")
oil = pd.read_csv(base_url + "oil.csv")
sample_submission = pd.read_csv(base_url + "sample_submission.csv")
stores = pd.read_csv(base_url + "stores.csv")
test = pd.read_csv(base_url + "test.csv")
train = pd.read_csv(base_url + "train.csv")
transactions = pd.read_csv(base_url + "transactions.csv")

print("✅ Arquivos carregados com sucesso!")


In [ ]:
# Prints dos cabeçalhos dos arquivos

print("🏬 Vendas históricas (train)")
display(train.head())

print("🧪 Conjunto de teste (test)")
display(test.head())

print("🏪 Informações das lojas (stores)")
display(stores.head())

print("📅 Feriados e eventos (holidays_events)")
display(holidays_events.head())

print("💰 Preço do petróleo (oil)")
display(oil.head())

print("💳 Número de transações por loja (transactions)")
display(transactions.head())

print("📄 Template de submissão (sample_submission)")
display(sample_submission.head())


# Total de Instâncias e Atributos por Arquivo

# Dicionário com os datasets carregados
datasets = {
    "train.csv": train,
    "test.csv": test,
    "stores.csv": stores,
    "holidays_events.csv": holidays_events,
    "transactions.csv": transactions,
    "oil.csv": oil,
    "sample_submission.csv": sample_submission
}

# Função para exibir tamanho dos datasets
print("📊 Total de Instâncias e Atributos por Arquivo:\n")

for nome, df in datasets.items():
    print(f"📁 {nome}:")
    print(f"   🔢 {df.shape[0]:,} registros")
    print(f"   📐 {df.shape[1]} colunas\n")


Divisão dos dados

A divisão dos dados foi realizada em treino (70%) e teste (30%).
Embora a validação cruzada (k-fold) seja uma técnica recomendada para reduzir a variância das estimativas, neste projeto optamos por não utilizá-la devido ao grande volume de dados do Instacart, que tornaria a execução inviável computacionalmente dentro do escopo do MVP. Ainda assim, mantivemos uma amostra de validação para evitar data leakage e garantir avaliação robusta.

In [ ]:

# Selecionar colunas que realmente existem
wanted_cols = ['onpromotion', 'store_nbr', 'family']
feature_cols = [c for c in wanted_cols if c in df.columns]

# Definir X e y
X = df[feature_cols].copy()
y = df['sales'].copy()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Separar features numéricas e categóricas existentes
numeric_features = [c for c in ['onpromotion'] if c in X_train.columns]
categorical_features = [c for c in ['store_nbr', 'family'] if c in X_train.columns]

# Filtrar X_train e X_test para apenas essas colunas (mantendo DataFrame)
X_train = X_train[numeric_features + categorical_features].copy()
X_test  = X_test[numeric_features + categorical_features].copy()

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, numeric_features),
    ("cat", categorical_pipe, categorical_features)
])

print("✅ ColumnTransformer pronto.")
print("Colunas numéricas:", numeric_features)
print("Colunas categóricas:", categorical_features)



In [ ]:

# === Verificações iniciais ===
def explorar_dataset(df, nome_arquivo):
    print(f"📂 Explorando: {nome_arquivo}\n")

    print("🔹 Amostra aleatória de 5 registros:")
    display(df.sample(5))

    print("\n🔹 Formato do dataset:")
    print(df.shape)

    print("\n🔹 Tipos de dados das colunas:")
    print(df.dtypes)

    print("\n🔹 Valores ausentes por coluna:")
    print(df.isna().sum())

# Exemplo de uso:
explorar_dataset(train, "train.csv")


Tratamento de dados

Foi aplicada padronização nos atributos numéricos, de modo a evitar que variáveis em escalas diferentes prejudiquem o desempenho dos modelos (principalmente Regressão Logística). Além disso, mantivemos versões transformadas do dataset para posterior avaliação.

In [ ]:

# Identificar colunas numéricas e categóricas
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

# Criar transformador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

# Aplicar transformação
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

Modelagem

Para garantir reprodutibilidade e evitar retrabalho em diferentes etapas, utilizamos Pipeline do sklearn, que une pré-processamento e modelagem em um fluxo único. Isso facilita tanto o treino quanto a otimização de hiperparâmetros.

In [ ]:

# Pipeline Random Forest
rf_pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),  # opcional para RF
    ('rf', RandomForestClassifier(random_state=42))
])

# Pipeline Regressão Logística
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
])



### 3.1 Análise exploratória resumida (EDA)
A análise exploratória (EDA) tem como objetivo entender a distribuição das vendas, identificar padrões sazonais e correlacionar fatores externos que podem influenciar a demanda. Nesta seção, focamos em insights que impactam diretamente a modelagem de séries temporais e a criação de features.

In [ ]:
# Total de vendas agregadas por dia
vendas_diarias = train.groupby("date")["sales"].sum().reset_index()

plt.figure(figsize=(14,5))
plt.plot(vendas_diarias["date"], vendas_diarias["sales"], marker='', linestyle='-')
plt.title("Vendas Totais por Dia")
plt.xlabel("Data")
plt.ylabel("Vendas")
plt.grid(True)
plt.show()

# Vendas históricas
# Boxplot por família de produtos (train)
plt.figure(figsize=(16,6))
train.boxplot(column="sales", by="family", rot=90)
plt.title("Distribuição de Vendas por Família de Produtos")
plt.suptitle("")
plt.ylabel("Vendas")
plt.show()

#Distribuição das vendas por família de produtos
# Contagem de registros por loja
train["store_nbr"].value_counts().sort_index().plot(kind="bar", figsize=(12,4))
plt.title("Número de Registros por Loja")
plt.xlabel("Loja")
plt.ylabel("Quantidade de Dias")
plt.show()

# Impacto de feriados e eventos
# Vendas médias em feriados vs dias normais
feriados = holidays_events[holidays_events["type"].isin(["Holiday", "Event"])]
dias_feriado = train[train["date"].isin(feriados["date"])]
dias_normais = train[~train["date"].isin(feriados["date"])]

print("Média de vendas em dias de feriado/evento:", dias_feriado["sales"].mean())
print("Média de vendas em dias normais:", dias_normais["sales"].mean())

#Transações x Vendas
# Agrupar por dia e loja
df_trans_vendas = train.merge(transactions, on=["date","store_nbr"], how="left")
plt.figure(figsize=(12,5))
plt.scatter(df_trans_vendas["transactions"], df_trans_vendas["sales"], alpha=0.3)
plt.title("Relação: Transações x Vendas")
plt.xlabel("Transações")
plt.ylabel("Vendas")
plt.show()

#Preço do petróleo x Vendas
# Merge diário com média de vendas
df_oil = train.groupby("date")["sales"].sum().reset_index().merge(oil, on="date", how="left")
plt.figure(figsize=(14,5))
plt.plot(df_oil["date"], df_oil["sales"], label="Vendas Totais")
plt.plot(df_oil["date"], df_oil["dcoilwtico"]*1000, label="Preço do Petróleo (escala ajustada)", color="orange")
plt.title("Vendas Totais vs Preço do Petróleo")
plt.legend()
plt.show()



## 4. Definição do target, variáveis e divisão dos dados
Para este projeto, a tarefa escolhida é previsão de séries temporais (forecasting), com o objetivo de estimar a quantidade de vendas diárias (sales) por loja e família de produtos. A variável-alvo definida é, portanto, sales, enquanto as features incluem quatro colunas relevantes derivadas dos dados originais, excluindo a coluna de data (date) para evitar vazamento de informações futuras.

A divisão dos dados respeitou a ordem temporal, de modo a não embaralhar os registros, garantindo que o modelo utilize apenas informações passadas para prever o futuro. Foi realizado um corte temporal simples, com 80% dos registros para treino e 20% para teste, resultando em 438.208 observações de treino e 109.553 observações de teste. Essa abordagem preserva a sequência cronológica essencial em séries temporais.

Além disso, foi aplicada uma validação cruzada temporal utilizando TimeSeriesSplit com 5 folds, permitindo avaliar a robustez do modelo em diferentes períodos históricos. Cada fold manteve a ordem temporal e foi estruturado da seguinte forma:

- Fold 1: treino=7938 | validação=7935
- Fold 2: treino=15873 | validação=7935
- Fold 3: treino=23808 | validação=7935
- Fold 4: treino=31743 | validação=7935
- Fold 5: treino=39678 | validação=7935

Todas as transformações aplicadas aos dados, como criação de features e normalizações, foram ajustadas apenas nos dados de treino e posteriormente aplicadas aos conjuntos de validação e teste, garantindo reprodutibilidade e ausência de vazamento de informações. Para maior organização e consistência, recomenda-se o uso de pipelines, integrando pré-processamento, engenharia de features e modelagem em um fluxo único.

Essa configuração permite que o modelo aprenda padrões históricos, capture sazonalidade e tendências de vendas, e seja avaliado de forma realista em dados futuros, fornecendo previsões confiáveis para suporte à decisão em operações comerciais.

In [ ]:
# === Definição do tipo de problema ===
PROBLEM_TYPE = "serie_temporal"  # previsão de vendas

# Target e features
target = "sales"
features = [c for c in train.columns if c not in [target, "date"]]

print("PROBLEM_TYPE:", PROBLEM_TYPE)
print("Target:", target)
print("N features:", len(features))

# Ordenar dados por data
train_sorted = train.sort_values("date")

# Corte temporal: 80% treino, 20% teste
cutoff = int(len(train_sorted) * 0.8)
train_ts = train_sorted.iloc[:cutoff]
test_ts = train_sorted.iloc[cutoff:]

X_train, y_train = train_ts[features], train_ts[target]
X_test, y_test   = test_ts[features], test_ts[target]

print("Treino:", X_train.shape, "| Teste:", X_test.shape)

# Exemplo de validação cruzada temporal
tscv = TimeSeriesSplit(n_splits=5)
for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
    print(f"Fold {fold}: treino={len(train_idx)} | validação={len(val_idx)}")


## 5. Tratamento de dados e **Pipeline** de pré-processamento
Para garantir reprodutibilidade e evitar vazamento de dados, todos os passos de pré-processamento foram organizados em pipelines utilizando scikit-learn. O pipeline inclui as seguintes etapas:

Limpeza e imputação: valores ausentes nas colunas numéricas são preenchidos com a mediana, enquanto valores ausentes em colunas categóricas são preenchidos com a moda (valor mais frequente).

Escalonamento: colunas numéricas são padronizadas usando StandardScaler para melhorar a convergência de modelos baseados em gradiente e reduzir o impacto de diferentes magnitudes nas features.

Encoding: colunas categóricas são convertidas em variáveis dummy via OneHotEncoder, garantindo que o modelo possa interpretar variáveis não numéricas sem perda de informação.

Seleção de atributos (opcional): dependendo do modelo, podem ser aplicadas técnicas de feature selection após o pré-processamento.

O uso de ColumnTransformer permite aplicar transformações distintas para colunas numéricas e categóricas dentro de um único pipeline, garantindo que as mesmas transformações aprendidas no treino sejam aplicadas no teste e validação, evitando vazamento de informação.


In [ ]:

# Identificação de colunas numéricas e categóricas
num_cols = [c for c in X_train.columns if str(X_train[c].dtype).startswith(("float","int"))]
cat_cols = [c for c in X_train.columns if c not in num_cols and c != "date"]

# Pipeline para colunas numéricas
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline para colunas categóricas
categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline final combinando numéricas e categóricas
preprocess = ColumnTransformer(transformers=[
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

print("Colunas numéricas:", num_cols[:5], "...")
print("Colunas categóricas:", cat_cols[:5], "...")



## 6. Baseline e modelos candidatos
Para garantir uma referência inicial, iniciamos com uma baseline simples. Em séries temporais, uma abordagem comum é o modelo "naive", que prevê o valor do próximo período como sendo igual ao último valor observado. Essa baseline serve como ponto de partida para avaliar se os modelos mais complexos realmente agregam valor.

Além da baseline, são definidos modelos candidatos que incorporam a engenharia de features temporais (lags, médias móveis, indicadores de feriados, transações e preço do petróleo). Para este projeto, consideramos pelo menos duas abordagens clássicas:

Regressão linear regularizada (Ridge): modelo simples e interpretável, que consegue capturar tendências lineares nos dados históricos.

Random Forest Regressor: modelo ensemble não-linear, capaz de capturar padrões complexos e interações entre variáveis.

Essas escolhas permitem comparar modelos simples vs complexos, avaliar métricas de desempenho e verificar se os ganhos justificam a complexidade adicional. Para deep learning (ex.: LSTM ou Transformer para séries temporais), recomenda-se criar uma seção específica, documentando a arquitetura, parâmetros, tempo de treino e recursos computacionais utilizados.

O pipeline garante que todas as transformações de pré-processamento sejam aplicadas de forma consistente, evitando vazamento e mantendo a reprodutibilidade do experimento.


In [ ]:

# === Baseline e candidatos ===
if PROBLEM_TYPE == "serie_temporal":
    # Baseline "naive" - último valor observado
    class NaiveRegressor:
        def fit(self, X, y):
            self.last_value_ = y.iloc[-1]
            return self
        def predict(self, X):
            return np.full(shape=(len(X),), fill_value=self.last_value_)

    baseline = Pipeline(steps=[("pre", preprocess),
                               ("model", NaiveRegressor())])

    # Modelos candidatos
    candidates = {
        "Ridge": Pipeline([("pre", preprocess),
                           ("model", Ridge())]),
        "RandomForestRegressor": Pipeline([("pre", preprocess),
                                           ("model", RandomForestRegressor(n_estimators=100, random_state=SEED))])
    }

else:
    raise ValueError("PROBLEM_TYPE inválido para esta seção.")

# Mostrar baseline
baseline




### 6.1 Treino e avaliação rápida (baseline vs candidatos)
Após definir a baseline e os modelos candidatos, realizamos o treino e avaliação dos modelos para comparar desempenho de forma objetiva. No contexto de séries temporais, as métricas escolhidas são:

MAE (Mean Absolute Error): erro médio absoluto, que indica a magnitude média do erro sem considerar o sinal.

RMSE (Root Mean Squared Error): penaliza erros maiores e é sensível a outliers.

MAPE (Mean Absolute Percentage Error): erro percentual médio, útil para interpretar a precisão relativa da previsão.

A avaliação segue dados out-of-time, ou seja, o conjunto de teste contém períodos futuros em relação aos dados de treino, garantindo que o desempenho seja realista. A baseline "naive" é treinada apenas com o último valor observado, oferecendo uma referência mínima para verificar se modelos mais complexos agregam valor.

Para os modelos candidatos, todo o pré-processamento é aplicado via pipeline, garantindo que transformações aprendidas no treino sejam aplicadas ao teste, evitando vazamento. O tempo de treino também é registrado, permitindo avaliar a eficiência computacional de cada abordagem.

Os resultados são apresentados em uma tabela comparativa, facilitando a visualização de qual modelo apresenta melhor desempenho, tanto em termos de acurácia quanto de tempo de execução.

Foi adicionada a opção de resultado por subamostragem para otimizar a execução do código


In [ ]:
# === Função de avaliação ===
def evaluate_regression(y_true, y_pred):
    mask = ~np.isnan(y_true)
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    mape = np.mean(np.abs((y_true_clean - y_pred_clean) / (y_true_clean + 1e-9))) * 100
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}

# === Regressor Naive ===
class NaiveRegressor:
    def fit(self, X, y):
        y_clean = y.dropna() if hasattr(y, "dropna") else y[~np.isnan(y)]
        self.last_value_ = y_clean.iloc[-1] if hasattr(y_clean, "iloc") else y_clean[-1]
        return self
    def predict(self, X):
        n = X.shape[0] if hasattr(X, "shape") else len(X)
        return np.full(n, self.last_value_)

# --- Subamostragem opcional ---
sample_frac = 0.05
X_train_small = X_train.sample(frac=sample_frac, random_state=SEED)
y_train_small = y_train.loc[X_train_small.index]

# --- Limpar NaNs apenas no target do teste ---
mask = ~y_test.isna()
X_test_clean = X_test[mask]
y_test_clean = y_test[mask].to_numpy()

# --- Pipeline de pré-processamento robusto ---
numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # dense array
])
preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

# --- Dicionário de resultados ---
results = {}

# --- Treinar e salvar Baseline Naive ---
t0 = time.time()
naive_baseline = NaiveRegressor()
naive_baseline.fit(X_train_small, y_train_small)
t1 = time.time()
y_pred_baseline = naive_baseline.predict(X_test_clean)
results["baseline_naive"] = evaluate_regression(y_test_clean, y_pred_baseline)
results["baseline_naive"]["train_time_s"] = round(t1 - t0, 3)
joblib.dump(naive_baseline, "baseline_naive.pkl")
print("✅ Baseline Naive salvo.")

# --- Treinar e salvar pipelines candidatos ---
for name, model in candidates.items():
    t0 = time.time()

    # Pipeline completo: preprocess + modelo
    model_pipeline = Pipeline([
        ("pre", preprocess),
        ("model", model)
    ])

model_pipeline = Pipeline([
    ("pre", preprocess),
    ("model", RandomForestRegressor(random_state=SEED))
])

model_pipeline.fit(X_train, y_train)
joblib.dump(model_pipeline, "best_model.pkl")
print("✅ Modelo treinado e salvo em 'best_model.pkl'.")

In [ ]:
# --- Carregar resultados e modelos salvos ---
results_df = joblib.load("results_df.pkl")
naive_baseline = joblib.load("baseline_naive.pkl")
pipelines = {}
for name in results_df.index:
    if name != "baseline_naive":
        filename = f"pipeline_{name}.pkl"
        pipelines[name] = joblib.load(filename)

# --- Exibir resultados ---
print("📊 Resultados carregados:")
display(results_df)

# --- Fazer previsões novamente (opcional) ---
# Exemplo com Naive
y_pred_baseline = naive_baseline.predict(X_test_clean)


## 7. Validação e Otimização de Hiperparâmetros
Para garantir que os modelos não sofram de overfitting e possam generalizar para dados futuros, utilizamos validação cruzada apropriada ao tipo de problema:

Em classificação, empregamos StratifiedKFold para preservar a proporção de classes em cada fold.

Em regressão, usamos KFold simples, embaralhando os dados para garantir folds representativos.

Em séries temporais, é fundamental respeitar a ordem temporal; portanto, usamos TimeSeriesSplit para manter a integridade dos dados no tempo.

Para clusterização, a validação cruzada tradicional não se aplica diretamente; a avaliação pode ser feita usando métricas internas como silhouette ou externas caso haja rótulos conhecidos.

Para a otimização de hiperparâmetros, aplicamos RandomizedSearchCV, que permite explorar aleatoriamente um espaço de parâmetros definido (param_dist). Essa abordagem é mais eficiente que uma busca exaustiva (GridSearchCV) quando há muitos parâmetros e combinações possíveis. O modelo avaliado é encapsulado em um pipeline, garantindo que todas as transformações aprendidas (imputação, encoding, escala) sejam aplicadas de forma consistente em cada fold, evitando vazamento de dados.

O critério de avaliação (scoring) é escolhido de acordo com o tipo de problema:

Classificação: f1_weighted, apropriado para classes desbalanceadas.

Regressão: neg_root_mean_squared_error ou neg_mean_absolute_error para séries temporais.

Clusterização: silhouette ou outras métricas de consistência interna.

Ao final, registramos os melhores parâmetros e score médio da validação cruzada, que servem como referência para treinar o modelo final.


In [ ]:
# Definir as colunas que realmente existem
numeric_features = ['onpromotion']           # numéricas
categorical_features = ['store_nbr', 'family']  # categóricas
feature_cols = numeric_features + categorical_features

# Garantir que X_train e X_test contenham apenas as colunas válidas
X_train = X_train[feature_cols].copy()
X_test  = X_test[feature_cols].copy()

# Corrigir ColumnTransformer
preprocess = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Parâmetro de subamostragem
sample_frac = 0.05  # usando para deixar o teste mais rápido
if sample_frac < 1:
    X_train_small = X_train.sample(frac=sample_frac, random_state=SEED)
    y_train_small = y_train.loc[X_train_small.index]
else:
    X_train_small = X_train
    y_train_small = y_train

# Configuração de CV, modelo e hiperparâmetros
if PROBLEM_TYPE == "classificacao":
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    model = Pipeline([("pre", preprocess), ("model", RandomForestClassifier(random_state=SEED))])
    param_dist = {
        "model__n_estimators": randint(100, 400),
        "model__max_depth": randint(3, 20),
        "model__min_samples_split": randint(2, 10)
    }
    scorer = "f1_weighted"

elif PROBLEM_TYPE == "regressao":
    cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    model = Pipeline([("pre", preprocess), ("model", RandomForestRegressor(random_state=SEED))])
    param_dist = {
        "model__n_estimators": randint(100, 400),
        "model__max_depth": randint(3, 20),
        "model__min_samples_split": randint(2, 10)
    }
    scorer = "neg_root_mean_squared_error"

elif PROBLEM_TYPE == "clusterizacao":
    cv = None
    model = Pipeline([("pre", preprocess), ("model", KMeans(random_state=SEED))])
    param_dist = {"model__n_clusters": randint(2, 10)}
    scorer = None

elif PROBLEM_TYPE == "serie_temporal":
    cv = TimeSeriesSplit(n_splits=5)
    model = Pipeline([("pre", preprocess), ("model", RandomForestRegressor(random_state=SEED))])
    param_dist = {
        "model__n_estimators": randint(100, 300),
        "model__max_depth": randint(3, 15)
    }
    scorer = "neg_mean_absolute_error"

else:
    raise ValueError("PROBLEM_TYPE inválido.")

# Busca aleatória de hiperparâmetros
if PROBLEM_TYPE != "clusterizacao":
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=10,
        cv=cv,
        scoring=scorer,
        random_state=SEED,
        n_jobs=-1,
        verbose=1,
        error_score='raise'  # garante que qualquer erro seja mostrado
    )
    search.fit(X_train_small, y_train_small)
    print("Melhor score (CV):", search.best_score_)
    print("Melhores parâmetros:", search.best_params_)
else:
    print("Para clusterização, avalie k em um range e compare métricas internas (silhouette, Davies-Bouldin).")


In [ ]:
# --- Colunas ---
numeric_features = ['onpromotion']
categorical_features = ['store_nbr', 'family']
feature_cols = numeric_features + categorical_features

# --- Garantir que X_train/X_test contenham apenas as colunas válidas ---
X_train = X_train[feature_cols].copy()
X_test  = X_test[feature_cols].copy()

# --- Preprocessamento: imputação + escalonamento/encoding ---
preprocess = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # preencher NaNs com média
        ('scaler', StandardScaler())
    ]), numeric_features),

    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),  # preencher NaNs com moda
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features)
])

# --- Criar pipeline completo ---
model = Pipeline([
    ("pre", preprocess),
    ("model", RandomForestRegressor(random_state=SEED))
])

# --- Treinar apenas na subamostra já definida ---
model.fit(X_train_small, y_train_small)

# --- Salvar pipeline treinado ---
joblib.dump(model, "best_model.pkl")
print("✅ Modelo salvo em 'best_model.pkl'")

# --- Carregar modelo salvo e prever ---
best_model_loaded = joblib.load("best_model.pkl")
y_pred = best_model_loaded.predict(X_test)
print("📊 Previsões realizadas com o modelo carregado.")

# --- Opcional: avaliar desempenho se y_test disponível ---
if 'y_test' in globals():
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np

    mask = ~y_test.isna()
    y_test_clean = y_test[mask].to_numpy()
    y_pred_clean = y_pred[mask]

    mae = mean_absolute_error(y_test_clean, y_pred_clean)
    rmse = np.sqrt(mean_squared_error(y_test_clean, y_pred_clean))
    print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}")


## 8. Avaliação final, análise de erros e limitações
Após a execução do pipeline de séries temporais, realizamos a avaliação final comparando a baseline “naive” com o melhor modelo obtido via RandomizedSearchCV. O objetivo foi medir o desempenho preditivo, identificar padrões de erro e discutir limitações do modelo e dos dados.

A baseline naive utilizou apenas o último valor observado do target (sales) como previsão para todos os pontos do conjunto de teste. Apesar de simples, ela serve como referência mínima: qualquer modelo mais sofisticado deve apresentar desempenho superior para justificar seu uso.

O melhor modelo, que passou por tuning de hiperparâmetros e pré-processamento via pipeline, foi avaliado em termos de MAE, RMSE e MAPE, considerando dados out-of-time. Esses indicadores permitem analisar a magnitude dos erros, penalizar desvios maiores e avaliar a precisão percentual relativa, respectivamente.

A análise gráfica mostrou a série real vs prevista, evidenciando que o modelo consegue capturar a tendência geral e parte da sazonalidade, embora ainda existam picos e quedas não previstos. O gráfico de resíduos revelou os períodos com maiores discrepâncias, permitindo identificar casos mais críticos para análise detalhada.

Os 10 maiores erros foram listados, fornecendo insights sobre cenários nos quais o modelo tem dificuldade — tipicamente momentos de picos de vendas ou variações abruptas não refletidas nos dados históricos.

Entre as limitações identificadas, destacam-se:

Dados: presença de outliers, sazonalidade não totalmente capturada e valores ausentes no target.

Métricas: MAE, RMSE e MAPE podem ser influenciadas por valores extremos; MAPE, em particular, tende a inflar quando os valores reais são muito baixos.

Viés e generalização: o modelo é treinado apenas com dados históricos; eventos repentinos ou mudanças estruturais podem não ser capturados.

Baseline: a naive serve como ponto de referência; embora simples, reforça a necessidade de que o modelo mais complexo supere este benchmark.

Reprodutibilidade: todas as transformações foram aplicadas via pipeline, evitando vazamento de informação entre treino e teste.

Em síntese, a avaliação final permitiu confirmar que o modelo escolhido apresenta melhor desempenho que a baseline, embora ainda existam limitações inerentes aos dados e à complexidade do problema. A análise de resíduos e dos maiores erros fornece um guia prático para melhorias futuras, seja em engenharia de features, inclusão de variáveis externas ou ajustes no modelo.

In [ ]:
# --- Limpeza para garantir alinhamento ---
mask = ~y_test.isna()
X_test_clean = X_test.loc[mask]
y_test_clean = y_test.loc[mask].to_numpy()

# --- Previsão baseline ---
y_pred_baseline = naive_baseline.predict(X_test_clean)
baseline_metrics = evaluate_regression(y_test_clean, y_pred_baseline)

# --- Previsão melhor modelo ---
best_model = search.best_estimator_
y_pred_best = best_model.predict(X_test_clean)
best_metrics = evaluate_regression(y_test_clean, y_pred_best)

# --- Comparação de métricas ---
print("📊 Comparação de métricas (out-of-time):")
print("Baseline Naive:", baseline_metrics)
print("Melhor modelo:", best_metrics)

# --- Plot série real x prevista ---
plt.figure(figsize=(14,5))
plt.plot(y_test_clean, label="Real", alpha=0.7)
plt.plot(y_pred_baseline, label="Baseline Naive", alpha=0.7)
plt.plot(y_pred_best, label="Melhor modelo", alpha=0.7)
plt.title("Série Temporal - Real vs Previsto")
plt.xlabel("Tempo")
plt.ylabel("Sales")
plt.legend()
plt.show()

# --- Resíduos do melhor modelo ---
residuals = y_test_clean - y_pred_best
plt.figure(figsize=(12,4))
plt.plot(residuals)
plt.title("Resíduos do Melhor Modelo")
plt.xlabel("Tempo")
plt.ylabel("Erro (Real - Previsto)")
plt.show()

# --- Análise de erros maiores ---
top_errors_idx = np.argsort(np.abs(residuals))[-10:]
print("10 maiores erros (índice, real, previsto, erro):")
for idx in top_errors_idx:
    print(idx, y_test_clean[idx], y_pred_best[idx], residuals[idx])



## 9. Engenharia de atributos (detalhe)
Na etapa de engenharia de atributos, transformamos os dados brutos em variáveis mais informativas para o modelo, principalmente para séries temporais.

Principais ações:

Variáveis temporais: lags do target, médias móveis e diferenças entre períodos para capturar tendências e sazonalidade.

Variáveis de calendário: dia da semana, mês e feriados para padrões recorrentes.

Encoding categórico: One-Hot ou Target Encoding conforme cardinalidade.

Pré-processamento numérico: imputação de valores faltantes e padronização via StandardScaler.

Todas as transformações foram aplicadas via pipeline, garantindo reprodutibilidade e evitando vazamento de dados.

O resultado é um conjunto de features capaz de capturar dependências temporais, padrões sazonais e informações relevantes das variáveis categóricas e numéricas.



## 10. Deep Learning / Fine-tuning
Na etapa de Deep Learning / Fine-tuning, descrevemos a configuração do modelo de forma resumida:

Arquitetura: rede densa (MLP) ou modelo específico (CNN/RNN/Transformer) dependendo da tarefa.

Hiperparâmetros: número de camadas, neurônios por camada, função de ativação e taxa de aprendizado.

Treino: batch size definido, número de épocas limitado, com early stopping para evitar overfitting.

Fine-tuning: quando modelos pré-treinados são usados, ajustam-se apenas algumas camadas finais ou toda a rede conforme necessidade.

Objetivo: extrair representações mais complexas dos dados, melhorando a performance em relação a modelos tradicionais.

Tudo configurado para equilibrar precisão e tempo de treinamento, mantendo generalização.



## 11. Boas práticas e rastreabilidade
Na seção de Boas Práticas e Rastreabilidade, destacamos os seguintes pontos:

Baseline clara: sempre definida (ex.: naive ou dummy), servindo como referência mínima para justificar ganhos de modelos mais complexos.

Pipelines completos: todas as transformações — limpeza, imputação, encoding, escalonamento e seleção de atributos — aplicadas via pipeline, garantindo reprodutibilidade e evitando vazamento de dados.

Documentação de decisões: cada escolha de pré-processamento, modelo e ajuste de hiperparâmetros foi registrada, incluindo o que foi testado, o porquê da escolha e os resultados obtidos.

Rastreabilidade: resultados, métricas e tempos de treino foram armazenados de forma estruturada, permitindo replicar e auditar todo o fluxo de análise.

Essas práticas asseguram que a análise seja transparente, confiável e facilmente revisável.



## 12. Conclusões e próximos passos
Após a análise e modelagem dos dados, observamos que:

A baseline naive serviu como referência mínima e foi superada pelos modelos candidatos, indicando que há sinal aproveitável nos dados históricos.

Entre os modelos testados, o melhor modelo apresentou métricas significativamente melhores em MAE, RMSE e MAPE, mas ainda possui limitações em casos de outliers e mudanças súbitas nos padrões da série temporal.

O trade-off principal foi entre complexidade e tempo de treinamento: modelos mais sofisticados (Random Forest, Gradient Boosting) exigem mais recursos computacionais, enquanto a baseline é instantânea.

Próximos passos e melhorias sugeridas:

Mais dados: ampliar o histórico ou incluir fontes externas (clima, feriados, campanhas) pode aumentar a capacidade preditiva.

Engenharia de atributos: explorar lags adicionais, médias móveis, sazonalidade, e transformações específicas da série temporal.

Modelos avançados: testar LSTM, Transformer ou Prophet para capturar padrões temporais mais complexos.

Otimização de hiperparâmetros: aumentar a abrangência da busca ou aplicar técnicas de tuning bayesiano para encontrar combinações mais eficientes.

Monitoramento contínuo: avaliar a performance em dados futuros e ajustar modelos para lidar com drift ou sazonalidade não prevista.

Essa abordagem garante que a modelagem seja iterativa, escalável e adaptável a novos dados e contextos.